# Process FINN and GFAS fire emissions datasets
 - Reads files
 - Plots gridded emissions
 - Generates NAME emission files

In [ ]:
import datetime
import numpy as np
import pandas as pd
import iris
import iris.plot as iplt
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import netCDF4 as nc
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature
from matplotlib.colors import ListedColormap, LinearSegmentedColormap


In [ ]:
PLOT_SPECIES = ['PM25Fire']

FINN_MODIS_ONLY = True
PM25FIRE_ONLY = True

# Path the FINN ascii file
if FINN_MODIS_ONLY:
    FINNPATH = '/data/users/bdrummon/FINN/FINNv2.5/2018/MODIS_only/FINNv2.5_mod_MOZART_2018_c20211213_subset.txt'
else:
    FINNPATH = '/data/users/bdrummon/FINN/FINNv2.5/2018/MODIS_VIIRS/FINNv2.5_modvrs_MOZART_2018_c20211213_subset.txt'


# Path to the GFAS netcdf file
GFASPATH = "/data/users/bdrummon/GFAS/*.nc"

# Path to a template NAME file
NAMEPATH = {
      "12km" : "/data/users/bdrummon/emissions/NAME/saddleworth_moor_2018/12km/gridded_emissions_snap01_201801160000.nc",
      "2p2km" : "/data/users/bdrummon/emissions/NAME/saddleworth_moor_2018/2p2km/gridded_emissions_snap01_201801160000.nc",
      "1km" : "/data/users/bdrummon/emissions/NAME/saddleworth_moor_2018/1km/gridded_emissions_snap01_201801160000.nc"
}

# Plot directory
if FINN_MODIS_ONLY:
    PLOTDIR = '/home/h01/bdrummon/plots/Saddleworth_Moor_New/FINN2.5/'
else:
    PLOTDIR = '/home/h01/bdrummon/plots/Saddleworth_Moor_New/FINN2.5_MODISVIIRS/'

# Directory to save emissions
EMISSIONS_SAVE_DIR = "/data/users/bdrummon/emissions/NAME/saddleworth_moor_2018"

# Setup the background map
BKMAP = cimgt.Stamen(style='terrain')

# Apply scaling factors (Graham et al)
APPLY_SCALING = False
# Extent of saddleworth moor fire to apply scaling factors to 
SCALING_DAYS = {
    177 : 5.,
    178 : 10.,
    179 : 10., 
    180 : 10.
}
# Bounding box for scaling factors - only Saddleworth Moor scaled
SMBOX = {
    "LONMIN" : -2.175,
    "LONMAX" : -1.90,
    "LATMIN" : 53.45,
    "LATMAX" : 53.61
}

# List of dates for SM fire higher emissions
SM_HIGH_EMS = [datetime.datetime(year=2018, month=6, day=day) for day in [26,27,28,29]]

# Start date for emissions generation
START = datetime.datetime(year=2018, month=6, day=14)
END = datetime.datetime(year=2018, month=7, day=17)

# Map NAME species to FINN
# Keys are FINN species
# Big alkenes (BIGENE) lumped into 1,3-butadiene
if PM25FIRE_ONLY:
    SPECIES_DICT = {
      'PM25' : 'PM25Fire'
                    }
else:
    SPECIES_DICT = {
        'CO' : 'CO',
        'NO' : 'NO',
        'NO2' : 'NO2',
        'SO2' : 'SULPHUR-DIOXIDE',
        'NH3' : 'AMMONIA',
        'PM25' : 'PM25',
        'PMC' : 'PMC',
        'BIGENE' : 'BD',
        'C2H4' : 'C2H4',
        'C3H6' : 'C3H6',
        'CH2O' : 'HCHO', 
        'CH3CHO' : 'CH3CHO',
        'ISOP' : 'C5H8',
        'MGLY' : 'MGLYOX',
        'TOLUENE' : 'TOLUEN',
        'XYLENE' : 'OXYL'
    }


GFAS_DICT = {
    'PM25' : 'Wildfire flux of Particulate Matter PM2.5',
    'CO' : 'Wildfire flux of Carbon Monoxide',
    'plume_top' : 'Altitude of plume top',
    'altitude_max_injection' : 'Mean altitude of maximum injection'
}

NAMES = {
    'CO' : 'carbon monoxide',
    'NO' : 'nitrogen monoxide',
    'NO2' : 'nitrogen dioxide',
    'SULPHUR-DIOXIDE' : 'sulphur dioxide',
    'AMMONIA' : 'ammonia',
    'PM25' : 'pm2p5 dry aerosol particles',
    'PMC' : 'pm coarse dry aerosol particles',
    'BD' : '1,3-butadiene',
    'C2H4' : 'ethene',
    'C3H6' : 'propene',
    'HCHO' : 'formaldehyde',
    'CH3CHO' : 'acetaldehyde',
    'C5H8' : 'isoprene',
    'MGLYOX' : 'methyl glyoxal',
    'TOLUEN' : 'toluene',
    'OXYL' : 'o-xylene',
    'PM25Fire' : 'pm2p5 dry aerosol particles from wildfire'
}

# Mole masses in g/mol
molar_masses = {
    ' APIN' : 136.23,
    'BENZENE' : 78.11,
    'BIGALK' : 72.151,
    'BIGENE' : 56.108, 
    'BPIN' : 136.238,
    'BZALD' : 106.124,
    'C2H2' : 26.038,
    'C2H4' : 28.051,
    'C2H6' : 30.070,
    'C3H6' : 42.081,
    'C3H8' : 44.097,
    'CH2O' : 30.026,
    'CH3CH2OH' : 46.069,
    'CH3CHO' : 44.053,
    'CH3CN' : 41.053,
    'CH3COCH3' : 58.080,
    'CH3COOH' : 60.052,
    'CH3OH' : 32.04,
    'CRESOL' : 108.13,
    'GLYALD' : 60.052,
    'HCN' : 27.0253,
    'HCOOH' : 46.025,
    'HONO' : 47.013,
    'HYAC' : 74.079,
    'ISOP' : 68.12,
    'LIMON' : 136.238,
    'MACR' : 70.09,
    'MEK' : 72.107,
    'MGLY' : 72.063,
    'MVK' : 70.09,
    'MYRC' : 136.238,
    'PHENOL' : 94.113,
    'TOLUENE' : 92.141,
    'XYLENE' : 106.168,
    'XYLOL' : 122.167,
    'CO' : 28.01,
    'NO' : 30.01,
    'NO2' : 46.0055,
    'SO2' : 64.066,
    'NH3' : 17.031
}

In [ ]:
# Load FINN ascii file and add datetime column
def load_finn():
    
    df = pd.read_csv(FINNPATH)
    # Add a datetime column from day of year
    df["DATE"] = [datetime.datetime.strptime(f"2018 {day}", "%Y %j") for day in df["DAY"]]

    return df

In [ ]:
# Load GFAS netcdf file and extract a geographical region
def load_gfas():
    
    # Bounding box
    lon = (-4, -1)
    lat = (52.5, 54.5)
    
    cubes = iris.load(GFASPATH)
    
    pdt1 = iris.time.PartialDateTime(year=2018,month=6,day=25)
    pdt2 = iris.time.PartialDateTime(year=2018,month=7,day=5)
    
    # Remove history attribute and concatenate
    for cube in cubes:
        cube.attributes['history'] = None
    cubes = cubes.concatenate()
    
    # Extract geographical region
    cubesout = iris.cube.CubeList([])
    for cube in cubes:
        cube = cube.intersection(longitude=lon)
        cube = cube.intersection(latitude=lat)
        cube = cube.extract(iris.Constraint(time=lambda cell: pdt1 < cell < pdt2))
        
        
        cubesout.append(cube)
    
    return cubesout

In [ ]:
def load_population(density=False):
    
    # Plot population exposure histogram
    variable_name = "UN WPP-Adjusted Population Count, v4.11 (2000, 2005, 2010, 2015, 2020): 2.5 arc-minutes"
    ds = nc.Dataset("/data/users/bdrummon/population_data/gpw_v4_population_count_adjusted_rev11_2pt5_min.nc")
    pop_count = ds.variables[variable_name][3,:,:]
    area = ds.variables[variable_name][8,:,:] + ds.variables[variable_name][8,:,:]
    lon = ds.variables['longitude'][:]
    lat = ds.variables['latitude'][:]

    # Create coordinates
    lon_coord = iris.coords.DimCoord(lon, standard_name='longitude', units='degrees', coord_system=iris.coord_systems.GeogCS(
        semi_major_axis=6378137.000,semi_minor_axis=6356752.314140))
    lat_coord = iris.coords.DimCoord(lat, standard_name='latitude', units='degrees', coord_system=iris.coord_systems.GeogCS(
        semi_major_axis=6378137.000,semi_minor_axis=6356752.314140))

    # Create cube
    pop = pop_count
    if density:
        pop = pop/area
        
    pop_cube = iris.cube.Cube(pop, var_name="pop", 
                             dim_coords_and_dims=[(lat_coord,0),(lon_coord,1)])

    return pop_cube

In [ ]:
# Function to process FINN data onto a grid, in a cube, and convert units
def process_finn(df, template_cube, species, day, surface = True):
    
    # Create a copy of the cube and initialise to zero
    cube = template_cube.copy()
    cube.data = np.zeros(cube.data.shape)
    
    if not df.empty:
        
        # Get latitudes and longitudes of grid
        gridlats = cube.coord('latitude').points
        gridlons = cube.coord('longitude').points

        # Iterate over rows in dataframe
        for index, row in df.iterrows():

            # Location of fire
            firelat = row['LATI']
            firelon = row ['LONGI']

            # Find nearest cell using cell centre
            idlat = (np.abs(gridlats - firelat)).argmin()
            idlon = (np.abs(gridlons - firelon)).argmin()

            # Calculate PM-Coarse 
            if species == "PMC":
                finn = row["PM10"] - row["PM25"]
            else:
                finn = row[species]

            # Grid cell areas
            cell_areas = iris.analysis.cartography.area_weights(cube[0])

            # Put the fire emissions into the cube at the right index
            # Put emissions into bottom vertical level
            if surface:
                idz = 0
            else:
                # Put the emissions into the appropriate level
                # Depends on day and location
                if ((SMBOX["LATMIN"] <= firelat <= SMBOX["LATMAX"]) &  
                    (SMBOX["LONMIN"] <= firelon <= SMBOX["LONMAX"])):
                    if day in SM_HIGH_EMS:
                        idz = 5
                    else:
                        idz = 4
                else:
                    idz = 6
                    
            cube.data[idz, idlat, idlon] += finn/cell_areas[idlat, idlon]                    
                    
        # Convert units to kg/m2/s
        # Gas species includes conversion from mole to kg
        cube.data = cube.data/24./60./60.
        if species not in ["PMC", "PM25"]:
            cube.data = cube.data*molar_masses[species] * 1e-3
    
    # Update time coord
    cube.coord('time').convert_units("days since 2018-01-01")
    cube.coord('time').points = np.asarray(day.timetuple().tm_yday-1) 
    
    # Set the attributes appropriately
    cube.attributes['tracer_name'] = SPECIES_DICT[species]
    cube.long_name = 'tendency of atmosphere mass content of '+NAMES[SPECIES_DICT[species]]+' due to emission'
    cube.var_name = SPECIES_DICT[species]
    cube.standard_name = None
    cube.attributes['emiss_sector'] = 'snap11'
    cube.attributes['source'] = 'FINN v2.4 MOZART-T1 speciation - Fire INventory from NCAR'
    cube.attributes['title'] = 'NAME emissions generated from FINN database'
    cube.attributes.pop('daily_scaling', None)
    cube.attributes.pop('hourly_scaling', None)
    cube.attributes.pop('vertical_scaling', None)
    
    return cube

In [ ]:
# Function to save cubelist 
def save_name_emissions_file(cubelist, grid, day, surface=False):
    
    # Name files are date ending 
    dt = day + datetime.timedelta(days=1)
    dt = dt.strftime("%Y%m%d")
    
    fname = "gridded_emissions_wildfire"
    
    if not FINN_MODIS_ONLY:
        fname = fname + "_modisviirs"
    if PM25FIRE_ONLY:
        fname = fname + "_pm25fire"
    fname = fname + f"_{dt}0000"
    
    if surface:
        directory = f"{grid}"
    else:
        directory = f"{grid}_high"
        
    iris.save(cubelist, f"{EMISSIONS_SAVE_DIR}/{directory}/{fname}.nc")
    print(f"  Saved file: {EMISSIONS_SAVE_DIR}/{directory}/{fname}.nc")

In [ ]:
# Make an emissions contour plots
def plot_emissions(cube, df=None, day=None, grid=None, species=None, model=None):
    
    setup_plot()
    
    pop_cube = load_population(density=True)
    pop_cube = pop_cube.extract(iris.Constraint(longitude=lambda cell: -3.1 < cell < -0.9))
    pop_cube = pop_cube.extract(iris.Constraint(latitude=lambda cell: 52.9 < cell < 54.1))
    iplt.contourf(pop_cube, cmap='Blues',zorder=10)
    cb = plt.colorbar(orientation='horizontal')
    cb.set_label("Population density [km$^{-2}$]")
    
    # Plot gridded emissions
    iplt.pcolormesh(cube, cmap=cmap, alpha=1, norm=colors.LogNorm(vmin=1e-9, vmax=2e-7), zorder=20)
    cb = plt.colorbar()
    cb.set_label(r"Emission flux [kg m$^{-2}$ s$^{-1}$]")
        
    # Calculate grid cell area and total emissions
    if cube.coord('latitude').bounds is None:
        cube.coord('latitude').guess_bounds()
    if cube.coord('longitude').bounds is None:
        cube.coord('longitude').guess_bounds()
        
    grid_area = iris.analysis.cartography.area_weights(cube)
    total = cube.collapsed(['latitude', 'longitude'], iris.analysis.SUM,  weights=grid_area)
        
    # Generate filename
    datetime = day.strftime("%Y%m%d")
    if grid is not None:
        fname = f"{species}_emission_{grid}_{datetime}"
    else:
        fname = f"{species}_emission_{datetime}"
    
    plt.tight_layout()
    plt.savefig(f"{PLOTDIR}{model}/{fname}.png", facecolor='white',dpi=300)
    plt.clf()

In [ ]:
Rearth = 6371.
norm = colors.LogNorm(vmin=1e-9, vmax=2e-7)

def plot_finn_native(df, day):
    
    setup_plot()
    ax = plt.gca()
    
    pop_cube = load_population(density=True)
    pop_cube = pop_cube.extract(iris.Constraint(longitude=lambda cell: -3.1 < cell < -0.9))
    pop_cube = pop_cube.extract(iris.Constraint(latitude=lambda cell: 52.9 < cell < 54.1))
    iplt.contourf(pop_cube, cmap='Blues',zorder=10)
    cb = plt.colorbar(orientation='horizontal')
    cb.set_label("Population density [km$^{-2}$]")
    
    latlons = []
    rates = []
    for index, row in df.iterrows():
        lat = row["LATI"]
        lon = row["LONGI"]
        # Check if lat-lon point already present
        if (lat,lon) in latlons:
            iloc = latlons.index((lat,lon))
            rates[iloc] = rates[iloc] + row["PM25"]/24./60./60./1000./1000.
        else:
        
            latlons.append((lat,lon))
            rates.append(row["PM25"]/24./60./60./1000./1000.)
        
    for latlon, rate in zip(latlons, rates):
        lat = latlon[0]
        lon = latlon[1]
        dlat = 1./111.195
        dlon = 2*np.pi*Rearth*np.cos(np.radians(lat))/360.
        dlon = 1./dlon
        print(rate, lat,lon)
        
        ax.add_patch(mpl.patches.Rectangle((lon-dlon/2.,lat-dlat/2.),dlon,dlat, ec='k',linewidth=0.5,fc=cmap(norm(rate)), zorder=200))
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cb = plt.colorbar(sm)
    cb.set_label(r"Emission flux [kg m$^{-2}$ s$^{-1}$]")

    
    #cb.set_label(r"Emission flux [kg m$^{-2}$ s$^{-1}$]")

        
    # Generate filename
    datetime = day.strftime("%Y%m%d")
    fname = f"PM25_emission_finn_native_{datetime}"
    
    plt.tight_layout()
    model="FINN"
    plt.savefig(f"{PLOTDIR}{model}/{fname}.png", facecolor='white',dpi=300)
    plt.clf()

In [ ]:
# Reds
NLEVELS = 9
colours = [[255,247,236],
           [254,232,200],
           [253,212,158],
           [253,187,132],
           [252,141,89],
           [239,101,72],
           [215,48,31],
           [179,0,0],
           [127,0,0]]
cmap = LinearSegmentedColormap.from_list("mycmap", np.array(colours)/255.)

def setup_plot():
    
    Locations = {"SM" : [53.543333, -1.955833],
                 "WH" : [53.63, -2.515]}
    
    Urban = {
        "Manchester" : [53.479444, -2.245278],
        "Salford" : [53.483, -2.2931],
        "Bolton" : [53.578, -2.429],
        "Warrington" : [53.38957, -2.590897],
        "Bury" : [53.593, -2.298],
        "Blackburn" : [53.748, -2.482 ],
        "Stockport" : [53.4083, -2.1494 ],
        "Preston" : [53.759, -2.699],
        "Oldham" : [53.5444, -2.1169]
    }

    reader = shpreader.Reader('/data/users/bdrummon/Shapefiles/UKCounties/Counties_and_Unitary_Authorities_(December_2016)_Boundaries/Counties_and_Unitary_Authorities_(December_2016)_Boundaries.shp')
    counties = list(reader.geometries())
    COUNTIES = cfeature.ShapelyFeature(counties, ccrs.PlateCarree())

    ax = plt.axes(projection=ccrs.PlateCarree())

    # Set extent and add county lines and locations
    ax.set_extent([-2.8,-1.85,53.3,53.8])    
    #ax.set_extent([-3.8, -1.5, 53.1, 54.3])
    ax.add_feature(COUNTIES, facecolor='none', edgecolor='gray',alpha=0.7,zorder=30)
#     for location in Locations:
#         plt.scatter(Locations[location][1],Locations[location][0], marker='^', color='k', s=20,zorder=50)
#         plt.text(Locations[location][1]+0.02,Locations[location][0]+0.02,location, fontsize=8,zorder=100)
    for site in Urban:
        plt.scatter(Urban[site][1],Urban[site][0], marker='*', color='k', s=20,zorder=50)
        if site == "Manchester":
            plt.text(Urban[site][1]+0.02,Urban[site][0],site, fontsize=6,zorder=100)
        elif site == "Oldham":
            plt.text(Urban[site][1]+0.01,Urban[site][0]+0.02,site, fontsize=6,zorder=100)
        else:
            plt.text(Urban[site][1]+0.01,Urban[site][0]+0.01,site, fontsize=6,zorder=100)
            

In [ ]:
# Make plots of heights from GFAS
def plot_gfas_heights(cube, variable=None, day=None, levels=None):
    
    # Set up axes
    ax = plt.axes(projection=BKMAP.crs)
    ax.set_extent((-2.7,-1.85,53.3,53.7), ccrs.PlateCarree())
    
    cmap = plt.cm.get_cmap('viridis')
    iplt.pcolormesh(cube, alpha=0.8, vmin=0., vmax=levels[-1], 
                    norm = colors.BoundaryNorm(levels, ncolors=cmap.N))
    cb = plt.colorbar()
    cb.set_label(f" [m]")
    
    # Add background map
    ax.add_image(BKMAP, 10, interpolation='spline36')
    
    plt.title(f"{day.strftime('%d/%m/%Y')}")
              
        # Generate filename
    datetime = day.strftime("%Y%m%d")

    fname = f"{variable}_{datetime}"
        
    plt.savefig(f"{PLOTDIR}GFAS/{fname}.png", dpi=150)
    plt.clf()

In [ ]:
save = False
plot = True

# Load FINN ascii file
df = load_finn()

# List of fire days
fire_days = pd.to_datetime(df["DATE"]).to_list()

# Loop over grids
for grid in ["1km","12km", "2p2km", "1km"]:
    print("Grid: ", grid)
    
    # Load NAME emissions file to act as template cube
    # Just take the first cube in the list 
    name_cube = iris.load(NAMEPATH[grid])[0]

    # List of datetimes to process
    days = pd.date_range(start=START,end=END).to_pydatetime().tolist()

    # Loop over all required days
    for day in days:
        print("Date: ", day)

        # Get fire emissions for this day
        dfday = df[df["DATE"]==day]

        # Process the fire emissions onto a grid
        for surface in [True, False]:
            cubelist = iris.cube.CubeList([])
            for species in SPECIES_DICT:
                cubelist.append(process_finn(dfday, name_cube, species, day, surface=surface))

            # Save NAME emissions file
            if save:
                save_name_emissions_file(cubelist, grid, day, surface=surface)
                
        
            # Plot emissions
            if plot:
                if surface:
                    if day in fire_days:
                        for species in PLOT_SPECIES:
                            cube = cubelist.extract_cube(iris.AttributeConstraint(tracer_name=species))
                            cube = cube.extract(iris.Constraint(**{"emissions layer number" : 1}))
                            plot_emissions(cube, df=dfday, day=day, grid=grid, species=species, model='FINN')
                            plot_finn_native(dfday, day)


In [ ]:
surface = True

# Code to write NAME sources block from FINN 
# Load FINN ascii file
df = load_finn()

if FINN_MODIS_ONLY:
    fout = open("/data/users/bdrummon/Saddleworth_Moor/NAME_input_file/Sources_MODIS.txt", 'w')
    if not surface:
        fout = open("/data/users/bdrummon/Saddleworth_Moor/NAME_input_file/Sources_MODIS_high.txt", "w")
else:
    fout = open("/data/users/bdrummon/Saddleworth_Moor/NAME_input_file/Sources_MODISVIIRS.txt", 'w')

# Create list of unique source locations
sources = {}
locations = []
for index, row in df.iterrows():
    date = row["DATE"]
    location = (row["LATI"], row["LONGI"])
    if location not in locations:
        locations.append(location)
    strength = row["PM25"] * 1000. / 24.
    if date in sources:
        # Add to existing 
        if location in sources[date]:
            # Add to existing
            sources[date][location] += strength
        else:
            # Add new location for this day
            sources[date][location] = strength
    else:
        # Add new source and location for this day
        sources[date] = {location : strength}
    
# # Locations block
fout.write("Locations: Release Locations\n")
fout.write("Name, H-Coord, X, Y\n")
for i, location in enumerate(locations):
    lat = round(location[0], 3)
    lon = round(location[1], 3)
    fout.write(f"Source{i}Loc, Lat-Long, {lon}, {lat}\n")
fout.write("\n")

# # Time dependency block
# # Hour scaling factors
factors = [0.13682737]*9 + [0.48009602, 0.96019204, 
                            1.68033607, 2.4004801, 
                            3.12062412, 3.84076815,
                            4.08081616, 2.88057612,
                            1.68033607, 0.96019204] \
            + [0.13682737]*5
fout.write("Source Time Dependency: Wildfire\n")
fout.write("From 1        , To 1          , Factor\n")
for i in range(24):
    if i == 23:
        end = 0
    else:
        end = i + 1
    fout.write(f"*/*/* {i:02}:00, */*/* {end:02}:00, {factors[i]}\n")
fout.write("\n")

# # Sources block
fout.write("Sources: \n")
fout.write("Name, Shape, H-Coord, Z-Coord, Set of Locations, Location, Z, dX, dY, dZ, dH-Metres?, dZ-Metres?, Angle, Source Strength, Time Dependency, Plume Rise?, Temperature, Flow Velocity, # Particles,  Max Age, Top Hat, Start Time, Stop Time, Lagrangian-Eulerian Time\n") 
i = 0
for date in sources:
    for location in sources[date]:
        
        
        # Put the fire emissions into the cube at the right index
        # Put emissions into bottom vertical level
        if surface:
            idz = 0
        else:
            
            # Put the emissions into the appropriate level
            # Depends on day and location
            if ((SMBOX["LATMIN"] <= location[0] <= SMBOX["LATMAX"]) &  
                (SMBOX["LONMIN"] <= location[1] <= SMBOX["LONMAX"])):
                if date in SM_HIGH_EMS:
                    idz = 5
                else:
                    idz = 4
            else:
                idz = 6
                
        if idz == 0:
            dz = 20.
            z1 = 10.
        elif idz == 4:
            dz = 198.
            z1 = 423.
        elif idz == 5:
            dz = 259.
            z1 = 651.
        elif idz == 6:
            dz = 325.
            z1 = 943.

        # Start and stop time
        start = date.to_pydatetime()
        stop = start + datetime.timedelta(days=1)
        start = start.strftime("%d/%m/%Y %H:%M")
        stop = stop.strftime("%d/%m/%Y %H:%M")
        
        iloc = locations.index(location)

        line = f"Source{i}, "
        line += "Cuboid, "
        line += "Lat-Long, "
        line += "m agl, "
        line += "Release Locations, "
        line += f"Source{iloc}Loc, "
        line += f"{z1}, "
        line += "1000., "
        line += "1000., "
        line += f"{dz}, "
        line += "Yes, "
        line += "Yes, "
        line += "0.0, "
        line += f"PM25Fire {sources[date][location]} g/hr, "
        line += "Wildfire, "
        line += "No, "
        line += "273.0, "
        line += "0.0, "
        line += "10000/hr, "
        line += "infinity, "
        line += "Yes, "
        line += f"{start}, "
        line += f"{stop}, "
        line += "infinity"

        fout.write(line+"\n")
        
        i += 1

fout.close()


In [ ]:
# Plot GFAS 
cubes = load_gfas()

# Emissions
for species in PLOT_SPECIES:
    cube = cubes.extract_cube(GFAS_DICT[species])
    for timeslice in cube.slices_over('time'):
        day = timeslice.coord('time').cell(0).point
        plot_emissions(timeslice, day=day, species=species, model='GFAS')

    
# These are height levels used for emissions by default in NAME and UKCA.
levels = {
    'altitude_max_injection' : [0., 20., 92., 184., 324., 522., 781., 1106],
}
# Heights
for variable in levels:
    cube = cubes.extract_cube(GFAS_DICT[variable])
    for timeslice in cube.slices_over('time'):
        day = timeslice.coord('time').cell(0).point
        plot_gfas_heights(timeslice, day=day, variable=variable, levels=levels[variable])